In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class WeatherDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_files = os.listdir(data_dir)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize the image to 224x224
            transforms.ToTensor(),  # Convert to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.data_dir, image_name)
        image = Image.open(image_path).convert('RGB')
        image_tensor = self.transform(image)
        # Extract label information
        label_name = image_name.split('.')[0]  # Remove file extension
        label = self.get_label(label_name)

        return image_tensor, label

    def get_label(self, label_name):
        # Return one-hot encoded label based on label name
        if label_name.startswith('Cloudy'):
            return torch.tensor([1, 0, 0, 0, 0])
        elif label_name.startswith('Foggy'):
            return torch.tensor([0, 1, 0, 0, 0])
        elif label_name.startswith('Rainy'):
            return torch.tensor([0, 0, 1, 0, 0])
        elif label_name.startswith('Snowy'):
            return torch.tensor([0, 0, 0, 1, 0])
        elif label_name.startswith('Sunny'):
            return torch.tensor([0, 0, 0, 0, 1])
        else:
            raise ValueError('Invalid label name')

# Set training data directory
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/5002/data/Data_Q2/train_data'

# Create dataset
dataset = WeatherDataset(train_data_dir)

# Set batch size and shuffle
batch_size = 32
shuffle = True

# Create data loader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [6]:
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Define the convolutional neural network model
class WeatherClassifier(nn.Module):
    def __init__(self, num_classes):
        super(WeatherClassifier, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.base_model(x)
        return x

# Set training parameters
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = WeatherClassifier(num_classes=5)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(num_epochs):
    model.train()  # Make sure the model is in training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, torch.argmax(labels, dim=1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

        running_loss += loss.item()

    # Print loss and accuracy for each epoch
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = correct / total * 100
    print(f"Epoch [{epoch+1}/{num_epochs}]\t Loss: {epoch_loss:.4f}\t Accuracy: {epoch_acc:.2f}%")

# Evaluate the model on the entire training dataset
model.eval()  # Make sure the model is in evaluation mode
total = 0
correct = 0

with torch.no_grad():
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

# Calculate accuracy on the training set
accuracy = correct / total * 100
print(f"\nAccuracy on training set: {accuracy:.2f}%")

Epoch [1/50]	 Loss: 0.8685	 Accuracy: 67.20%
Epoch [2/50]	 Loss: 0.5025	 Accuracy: 84.40%
Epoch [3/50]	 Loss: 0.2452	 Accuracy: 89.20%
Epoch [4/50]	 Loss: 0.1419	 Accuracy: 94.40%
Epoch [5/50]	 Loss: 0.1959	 Accuracy: 95.20%
Epoch [6/50]	 Loss: 0.2013	 Accuracy: 92.40%
Epoch [7/50]	 Loss: 0.1565	 Accuracy: 94.80%
Epoch [8/50]	 Loss: 0.0788	 Accuracy: 96.80%
Epoch [9/50]	 Loss: 0.1003	 Accuracy: 96.80%
Epoch [10/50]	 Loss: 0.1160	 Accuracy: 96.80%
Epoch [11/50]	 Loss: 0.1152	 Accuracy: 96.00%
Epoch [12/50]	 Loss: 0.1153	 Accuracy: 95.60%
Epoch [13/50]	 Loss: 0.0863	 Accuracy: 97.20%
Epoch [14/50]	 Loss: 0.0522	 Accuracy: 98.00%
Epoch [15/50]	 Loss: 0.0443	 Accuracy: 97.60%
Epoch [16/50]	 Loss: 0.0192	 Accuracy: 99.20%
Epoch [17/50]	 Loss: 0.1000	 Accuracy: 96.80%
Epoch [18/50]	 Loss: 0.0552	 Accuracy: 97.60%
Epoch [19/50]	 Loss: 0.0469	 Accuracy: 98.40%
Epoch [20/50]	 Loss: 0.0730	 Accuracy: 97.60%
Epoch [21/50]	 Loss: 0.0684	 Accuracy: 98.00%
Epoch [22/50]	 Loss: 0.0536	 Accuracy: 97.6

In [7]:
model = WeatherClassifier(num_classes=5)
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        print(f"{name}: {module}")
        #print(f"Input Shape: {list(module.weight.shape)}")
        if module.bias is not None:
            print(f"Output Shape: {list(module.bias.shape)}")

base_model.conv1: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
base_model.layer1.0.conv1: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
base_model.layer1.0.conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
base_model.layer1.1.conv1: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
base_model.layer1.1.conv2: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
base_model.layer2.0.conv1: Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
base_model.layer2.0.conv2: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
base_model.layer2.0.downsample.0: Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
base_model.layer2.1.conv1: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
base_model.layer2.1.conv2: Conv2d(128, 128, kernel_size=(3, 3), stri